# 역진자 태스크 "CartPole"
- 역진자 태스크란 수레 위에 회전축을 고정한 봉을 세워두고, 수레를 좌우로 움직이며 이 봉이 쓰러지지 않도록 제어하는 과제를 말한다.

In [15]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [16]:
# 애니메이션을 만드는 함수
# 참고 URL http://nbviewer.jupyter.org/github/patrickmineault/xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display


def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    #anim.save('movie_cartpole.mp4')  # 주석 추가 : 애니메이션을 저장하는 부분
    display(display_animation(anim, default_mode='loop'))

In [17]:
# 수레를 무작위로 움직임

frames = []
env = gym.make('CartPole-v0')
observation = env.reset()  # 먼저 환경을 초기화해야 함

for step in range(0, 200):
    frames.append(env.render(mode='rgb_array'))  # frames에 각 시각의 이미지를 추가한다
    action = np.random.choice(2)  # 0(수레를 왼쪽으로), 1(수레를 오른쪽으로) 두 가지 행동을 무작위로 취함
    observation, reward, done, info = env.step(action)  # action을 실행

# 주의: 실행했을 때 ipykernel_launcher.p... 라는 창이 나타날 수 있지만, 무시하면 된다

/home/user/anaconda3/envs/Pytorch/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [18]:
# 애니메이션을 파일로 저장하고 재생함
display_frames_as_gif(frames)

/home/user/anaconda3/envs/Pytorch/lib/python3.8/site-packages/JSAnimation/html_writer.py:281: MatplotlibDeprecationWarning: 
The 'clear_temp' parameter of setup() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. If any parameter follows 'clear_temp', they should be passed as keyword, not positionally.
  super(HTMLWriter, self).setup(fig, outfile, dpi,


AttributeError: 'HTMLWriter' object has no attribute '_temp_names'

----------------------------------------------

# 다변수 연속 값을 이산변수로 변환하기

In [19]:
# 구현에 사용할 패키지 임포트
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [20]:
# 상수 정의
ENV = 'CartPole-v0'  # 태스크 이름
NUM_DIZITIZED = 6  # 각 상태를 이산변수로 변환할 구간 수

In [21]:
# CartPole 실행
env = gym.make(ENV)  # 태스크 실행 환경 생성
observation = env.reset()  # 환경 초기화

In [22]:
# 이산변수 변환에 사용할 구간을 계산
def bins(clip_min, clip_max, num):
    '''관측된 상태(연속값)을 이산값으로 변환하는 구간을 계산'''
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

In [24]:
np.linspace(-2.4, 2.4, 6 + 1) # linspace는 나눌 구간의 수를 인자로 받아 해당 데이터의 범위를 구간 수에 맞게 나누어주는 함수이다.

array([-2.4, -1.6, -0.8,  0. ,  0.8,  1.6,  2.4])

In [25]:
np.linspace(-2.4, 2.4, 6 + 1)[1:-1]

array([-1.6, -0.8,  0. ,  0.8,  1.6])

In [29]:
def digitize_state(observation):
    '''관측된 상태(observation)을 이산값으로 변환'''
    cart_pos, cart_v, pole_angle, pole_v = observation
    # dititize함수의 경우 bins 인자를 통해 연속된 값을 구간의 수로 나누었을 때 인자로 받은 값이 어느 범위의 이산 변수값을 확인하여 이산변수를 반환해주는 함수이다.
    digitized = [
        np.digitize(cart_pos, bins=bins(-2.4, 2.4, NUM_DIZITIZED)), # cart_pos가 bins인자로 받은 구간에서 어디에 속하는지를 이산 값으로 반환 
        np.digitize(cart_v, bins=bins(-3.0, 3.0, NUM_DIZITIZED)),  # cart_v 가 bins인자로 받은 구간에서 어디에 속하는지를 이산 값으로 반환
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, NUM_DIZITIZED)), # pole_angle 가 bins인자로 받은 구간에서 어디에 속하는지를 이산 값으로 반환
        np.digitize(pole_v, bins=bins(-2.0, 2.0, NUM_DIZITIZED))] # pole_v 가 bins인자로 받은 구간에서 어디에 속하는지를 이산 값으로 반환
    return sum([x * (NUM_DIZITIZED**i) for i, x in enumerate(digitized)]) # NUM_DIGITIZE진수로 변환하여 값을 반환함

In [30]:
digitize_state(observation)

741

-------------------------------------------------------------------------------

# Q-learning을 사용한 CartPole

In [50]:
# 구현에 사용할 패키지 임포트
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [88]:
# 애니메이션을 만드는 함수
# 참고 URL http://nbviewer.jupyter.org/github/patrickmineault
# /xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=50)

    # anim.save('movie_cartpole.mp4')  # 애니메이션을 저장하는 부분
    display(display_animation(anim, default_mode='loop'))

In [111]:
# 상수 정의
ENV = 'CartPole-v0'  # 태스크 이름
NUM_DIZITIZED = 8  # 각 상태를 이산변수로 변환할 구간 수
GAMMA = 0.99  # 시간할인율
ETA = 0.7  # 학습률
MAX_STEPS = 200  # 1에피소드 당 최대 단계 수
NUM_EPISODES = 1000  # 최대 에피소드 수

In [112]:
class Agent:
    '''CartPole 에이전트 역할을 할 클래스, 봉 달린 수레다.'''
    def __init__(self, num_states, num_actions):
        self.brain = Brain(num_states, num_actions) 
        
    def update_Q_function(self, observation, action, reward, observation_next): # Q함수 수정
        self.brain.update_Q_table(observation, action, reward, observation_next)
        
    def get_action(self, observation, step): # 행동결정
        action = self.brain.decide_action(observation, step)
        return action

In [113]:
class Brain:
    '''Agent의 두뇌 역할을 하는 클래스, Q러닝을 실제 수행'''
    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        # Create Q table
        self.q_table = np.random.uniform(low=0, high=1, size=(NUM_DIGITIZED**num_states, num_actions))
        
    def bins(self, clip_min, clip_max, num):
        return np.linspace(clip_min, clip_max, num+1)[1:-1] # 연속값을 이산 변수로 변환하는 구간을 계산
    
    def digitize_state(self, observation):
        cart_pos, cart_v, pole_angle, pole_v = observation # the shape of observation : (1, 4)
        digitized = [
            np.digitize(cart_pos, self.bins(-2.4, 2.4, NUM_DIGITIZED)),
            np.digitize(cart_v, self.bins(-3.0, 3.0, NUM_DIGITIZED)),
            np.digitize(pole_angle, self.bins(-0.5, 0.5, NUM_DIGITIZED)),
            np.digitize(pole_v, self.bins(-2.0, 2.0, NUM_DIGITIZED))
        ]
        return sum([x * (NUM_DIGITIZED**i) for i, x in enumerate(digitized)])
    
    def update_Q_table(self, observation, action, reward, observation_next):
        state = self.digitize_state(observation) # 현재 관찰 값에 대한 상태를 구함
        state_next = self.digitize_state(observation_next) # 다음 관찰 값에 대한 상태를 구함
        
        Max_Q_next = max(self.q_table[state_next][:]) # 최댓 값을 구함
        # 최댓값을 사용해서 q_table을 수정함
        self.q_table[state, action] = self.q_table[state, action] + ETA * (reward + GAMMA * Max_Q_next - self.q_table[state, action])
        
    def decide_action(self, observation, episode):
        '''epsilon-greedy 알고리즘을 적용해 서서히 최적 행동의 비중을 늘림'''
        state = self.digitize_state(observation)
        epsilon = 0.5 * (1 / (episode + 1)) # episode를 실행할수록 최적 행동의 비중을 늘림
        
        if epsilon <= np.random.uniform(0, 1):
            action = np.argmax(self.q_table[state][:])
        else:
            action = np.random.choice(self.num_actions)
        return int(action)

In [118]:
class Environment:
    '''CartPole을 실행하는 환경역할을 하는 클래스'''
    def __init__(self):
        self.env = gym.make(ENV)
        num_states = self.env.observation_space.shape[0] # 태스크의 상태 변수 수를 구함
        num_actions = self.env.action_space.n # 가능한 행동 수를 구함
        self.agent = Agent(num_states, num_actions)
        
    def run(self):
        complete_episode = 0 # 195단계 이상 버틴 에피소드 수
        is_episode_final = False # 마지막 에피소드 여부
        frames = []
        
        for episode in range(NUM_EPISODE):
            observation = self.env.reset()
            for step in range(MAX_STEPS):
                if is_episode_final is True:
                    frames.append(self.env.render(mode='rgb_array'))
                
                # 여기서 step인자로 episode를 선택한 이유는 epsilon-greedy알고리즘을 학습할때 episode가 진행될 때마다 epsilon을 줄이기 위함
                action = self.agent.get_action(observation, episode) # Select action
                observation_next, _, done, _ = self.env.step(action) # Calculate next state and next action
                
                if done:
                    if step < 195:
                        reward = -1 # 봉이 쓰러지면 페널티로 보상 -1을 부여
                        complete_episode = 0
                    else:
                        reward = 1 # 성공시 보상으로 1을 주어줌
                        complete_episode += 1 # 연속된 성공 기록을 업데이트
                else:
                    reward = 0 # episode가 진행되는 동안은 보상이 없다.
                
                # observation_next를 사용해 Q함수를 수정
                self.agent.update_Q_function(observation, action, reward, observation_next)
                # 다음 단계 상태 관측
                observation = observation_next
                
                if done:
                    print(f"{episode} Episode : Fidished after {(step + 1)} time steps")
                    break
                    
            if is_episode_final is True:
                display_frames_as_gif(frames)
                break
            
            if complete_episode >= 10:
                print("10 episode 연속 성공")
                is_episode_final = True # 다음 에피소드가 마지막 에피소드가 됨

In [119]:
cartpole_env = Environment()
cartpole_env.run()

0 Episode : Fidished after 17 time steps
1 Episode : Fidished after 57 time steps
2 Episode : Fidished after 38 time steps
3 Episode : Fidished after 20 time steps
4 Episode : Fidished after 18 time steps
5 Episode : Fidished after 44 time steps
6 Episode : Fidished after 12 time steps
7 Episode : Fidished after 17 time steps
8 Episode : Fidished after 12 time steps
9 Episode : Fidished after 8 time steps
10 Episode : Fidished after 9 time steps
11 Episode : Fidished after 34 time steps
12 Episode : Fidished after 11 time steps
13 Episode : Fidished after 24 time steps
14 Episode : Fidished after 11 time steps
15 Episode : Fidished after 25 time steps
16 Episode : Fidished after 35 time steps
17 Episode : Fidished after 9 time steps
18 Episode : Fidished after 29 time steps
19 Episode : Fidished after 31 time steps
20 Episode : Fidished after 34 time steps
21 Episode : Fidished after 45 time steps
22 Episode : Fidished after 14 time steps
23 Episode : Fidished after 8 time steps
24 Epi

/home/user/anaconda3/envs/Pytorch/lib/python3.8/site-packages/JSAnimation/html_writer.py:281: MatplotlibDeprecationWarning: 
The 'clear_temp' parameter of setup() was deprecated in Matplotlib 3.3 and will be removed two minor releases later. If any parameter follows 'clear_temp', they should be passed as keyword, not positionally.
  super(HTMLWriter, self).setup(fig, outfile, dpi,


AttributeError: 'HTMLWriter' object has no attribute '_temp_names'